In [1]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print("GPU is available for TensorFlow!")
else:
    print("No GPU found for TensorFlow.")


TensorFlow version: 2.18.0
GPU is available for TensorFlow!


In [2]:
import torch

print("PyTorch version:", torch.__version__)
if torch.cuda.is_available():
    print("GPU is available for PyTorch!")
else:
    print("No GPU found for PyTorch.")


PyTorch version: 2.5.1+cu124
GPU is available for PyTorch!


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {accuracy:.4f}')


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8760 - loss: 0.4393 - val_accuracy: 0.9590 - val_loss: 0.1373
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9652 - loss: 0.1247 - val_accuracy: 0.9683 - val_loss: 0.1027
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9777 - loss: 0.0761 - val_accuracy: 0.9737 - val_loss: 0.0905
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9820 - loss: 0.0569 - val_accuracy: 0.9750 - val_loss: 0.0845
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9873 - loss: 0.0421 - val_accuracy: 0.9747 - val_loss: 0.0804
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9717 - loss: 0.0941
Test accuracy: 0.9747


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Load and preprocess the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')


Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 9.91M/9.91M [00:02<00:00, 4.94MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 28.9k/28.9k [00:00<00:00, 131kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 1.65M/1.65M [00:06<00:00, 245kB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 4.54k/4.54k [00:00<00:00, 9.12MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/5], Loss: 0.1478
Epoch [2/5], Loss: 0.1069
Epoch [3/5], Loss: 0.0933
Epoch [4/5], Loss: 0.2779
Epoch [5/5], Loss: 0.0450
Test Accuracy: 96.55%


1. What is the purpose of normalizing the input data in both TensorFlow and PyTorch implementations?

Normalization guarantees that all input features are scaled to a comparable range (usually between 0 and 1 or standardized with a mean of 0 and a variance of 1). This aids in:

Accelerating convergence throughout the training process.

Avoiding problems associated with varying feature scales.

Minimizing the likelihood of disappearing and exploding gradients.

Enhancing model stability and effectiveness.



2.  Explain the role of the activation function relu in the neural network.
ReLU (Rectified Linear Unit) is an activation function that adds non-linearity to the model, enabling it to learn intricate patterns. It functions as:  

                                    f(x)=max(0,x)

x>0 

If x is less than or equal to 0, it results in 0.

Advantages of ReLU:

Prevents the vanishing gradient issue found in sigmoid or tanh activation functions.

Accelerates training by enabling effective backpropagation.

Assists the model in understanding intricate patterns more efficiently.



3. Why is it important to use GPU for training neural networks?

GPUs play a vital role in deep learning due to their:

Manage parallel calculations effectively, greatly accelerating matrix operations.

Shorten training duration from hours to minutes for extensive datasets.

Enable the training of deeper networks without significant delays.

Enhance memory utilization and enable training of bigger models.



4. Compare the training time and accuracy of the TensorFlow and PyTorch models. Which one performed better and why?
Comparison:

Precision: TensorFlow achieved a marginally higher performance (97.47%) than PyTorch (96.55%).

Loss: PyTorch achieved a final loss of 0.0450, which is lower than TensorFlow's 0.0941, suggesting superior optimization.

Training Duration: TensorFlow required approximately 5-7 seconds for each epoch. The training duration of PyTorch isn't apparent but is usually comparable.

Summary:

If precision is the main focus - TensorFlow outperformed others.

If greater model stability and reduced loss are prioritized - PyTorch showed superior performance.

